# Parallel runs on the PIK cluster (e.g. Monte Carlo runs)
The example script below shows how to run SWIM in parallel on the PIK cluster, using the example of a Monte Carlo random parameter setting. It makes use of the project *cloning* functionality of SWIMpy and SLURM `sbatch` job submission that is executed in the background.

In [16]:
# hidden cell: code to setup
import swimpy, os
%matplotlib inline

project_path = os.path.join(os.path.dirname(swimpy.__file__), '../tests/project')
os.chdir(project_path)
if not os.path.exists('swimpy'):
    p = swimpy.project.setup()

Initialising a new modelmanager project in: .
with settings file in: ./swimpy/settings.py


In [ ]:
import os, time
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

import swimpy

# number of runs
nruns = 10

# parameter and ranges (extend as needed)
parameter = {'ecal': (0.8, 1.2),
             'sccor': (0.1, 10),
             'smrate': (0.1, 0.6)}

# load the project instance
p = swimpy.Project()

# clone and submit
for i in range(nruns):
    name = 'monte_carlo_%02i' % i
    clone = p.clone(name, Browser=None)  # disable Browser
    # set random parameters
    rnp = {k: np.random.uniform(*b) for k, b in parameter.items()}
    clone.basin_parameters.update(rnp)
    # submit cluster run
    clone.run(cluster=name, tags='montecarlo %s' % name, files=['station_daily_discharge'])

# wait and get for runs
runs = p.browser.runs.wait(nruns, tags__contains='montecarlo')

# unpack station_daily_discharge
q = pd.DataFrame({r.id: r.station_daily_discharge['BLANKENSTEIN'] for r in runs})

The default SLURM settings may be extended like a normal dictionary. E.g.:

In [14]:
p.slurmargs
p.slurmargs.update({'qos': 'medium', 'time': '0:0:01'})
p.slurmargs

{'account': 'swim', 'qos': 'medium', 'time': '0:0:01'}